In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
train_path = "C:/Users/Revathi/OneDrive/Desktop/dataset/train"
validate_path = "C:/Users/Revathi/OneDrive/Desktop/dataset/validate"
test_path = "C:/Users/Revathi/OneDrive/Desktop/dataset/test"

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['apj abdul kalam','revathi'], batch_size=10)
validate_batches = ImageDataGenerator().flow_from_directory(validate_path, target_size=(224,224), classes=['apj abdul kalam','revathi'], batch_size=3)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['apj abdul kalam','revathi'], batch_size=3)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1]!=3):
            ims = ims.transpose((0,2,3,1))
    
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation = None if interp else 'none')

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
plots(imgs, titles=labels)

In [ ]:
#Build and train CNN
model = Sequential([
    Conv2D(32,(3,3), activation='relu', input_shape=(224,224,3)), Flatten(), 
    Dense(2, activation='softmax')
])

In [ ]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=4, validation_data=validate_batches, validation_steps=4, epochs=5, verbose=2)

In [ ]:
#predict
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)


In [ ]:
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
predictions

In [ ]:
cm = confusion_matrix(test_labels, predictions[:,0])

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print("Confusion Matrix, without normalization")
    
    print(cm)
    
    thresh = cm.max()/ 2
    for i, j in itertools.product(range(cm.shap[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
cm_plot_labels = ['apj abdul kalam','revathi']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
#Build Fine Tuned VGG16 model
vgg16_model = keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
type(vgg16_model)

In [ ]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
model.layers.pop()

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#Train the Fine Tuned vgg16 model 
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, steps_per_epoch=4, validation_data=validate_batches, validation_steps=4, epochs=5, verbose=2)

In [ ]:
#predict using fine tuned vgg16 model
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [ ]:
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))